In [7]:
from logging import error
from os import write
import re
from nbformat import read
from lxml import etree
from requests import Session
from requests.auth import HTTPBasicAuth
from zeep import Client, Settings, Plugin, helpers
from zeep.transports import Transport
from zeep.exceptions import Fault
import pandas as pd
import sys
from yachalk import chalk
import numpy as np
import logging
import logging.config
import json

In [8]:
logging.getLogger('zeep').setLevel(logging.DEBUG)
DEBUG = True
LEVEL = 'ERROR'
WSDL_FILE = 'AXLAPI.wsdl'
# dcloud UCM
user = 'administrator'
passwd = 'dCloud123!'
cucm = '198.18.133.3'

logging.config.dictConfig({
    'version': 1,
    'formatters': {
        'verbose': {
            'format': '%(name)s: %(message)s'
        }
    },
    'handlers': {
        'console': {
            'level': LEVEL,
            'class': 'logging.StreamHandler',
            'formatter': 'verbose',
        },
    },
    'loggers': {
        'zeep.transports': {
            'level': LEVEL,
            'propagate': True,
            'handlers': ['console'],
        },
    }
})

In [9]:
# AXl session initialisation
session = Session()
session.verify=False
session.auth = HTTPBasicAuth( user,passwd )
transport = Transport( session = session, timeout = 10 )
settings = Settings( strict = False, xml_huge_tree = True)
client = Client( WSDL_FILE, settings = settings, transport = transport)
service = client.create_service( '{http://www.cisco.com/AXLAPIService/}AXLAPIBinding',f'https://{cucm}:8443/axl/')

def read_input_data(file):
    data = pd.read_csv(file)
    return data

def get_phone(service, phone_name):
    phone  = service.getPhone(name=phone_name)
    phone_od = helpers.serialize_object(phone['return']['phone'])
    return phone_od

def refine_phone_input(phone_info):
    phone_info['name'] = 'SEP123212321299'
    phone_info['protocol'] = 'SIP'
    phone_info['securityProfileName'] = 'Cisco 8945 - Standard SIP Non-Secure Profile'
    phone_info['sipProfileName'] = 'Standard SIP Profile'
    phone_info['phoneTemplateName'] = 'Standard 8945 SIP'

    phone_info.pop('confidentialAccess')
    phone_info.pop('loadInformation')
    # phone_info.pop('preemption')
    phone_info.pop('mlppIndicationStatus')
    # phone_info.pop('singleButtonBarge')
    phone_info.pop('vendorConfig')
    phone_info.pop('currentConfig')
    # phone_info.pop('ctiid')
    # phone_info.pop('uuid')
    #phone_info.pop('versionStamp')
    phone_info.pop('mraServiceDomain')

    # new_phone_info = {
    #     'name': 'SEP123212321299',
    #     'protocol': 'SIP',
    #     'model': 'Cisco 8945',
    #     'product': 'Cisco 8945',
    #     'class': 'Phone',
    #     'protocolSide': 'User',
    #     'callingSearchSpaceName': phone_info['callingSearchSpaceName'],
    #     'devicePoolName':phone_info['devicePoolName'],
    #     'lines':phone_info['lines'],
    #     'commonPhoneConfigName':phone_info['commonPhoneConfigName'],
    #     'locationName':phone_info['locationName'],
    #     'useTrustedRelayPoint':phone_info['useTrustedRelayPoint'],
    #     'phoneTemplateName':'Standard 8945 SIP',
    #     'primaryPhoneName': None,
    #     'sipProfileName': 'Standard SIP Profile',
    #     'securityProfileName': 'Cisco 8945 - Standard SIP Non-Secure Profile',
    #     'builtInBridgeStatus': phone_info['builtInBridgeStatus'],
    #     'packetCaptureMode':phone_info['packetCaptureMode'],
    #     'certificateOperation': phone_info['certificateOperation'],
    #     'deviceMobilityMode': phone_info['deviceMobilityMode'],
    #     'commonDeviceConfigName': None
    # }
    #print(new_phone_info)
    return phone_info

def add_phone(service, phone_config, index,logs):
    try:
        service.addPhone(phone_config)
        logs.loc[index,'Status'] = "Migration Succeeded"
        print("Phone {} migrated successfully".format(phone_config['name']))
    except Fault as err:
        print("{} Add SIP Phone failed. Error: {}".format(phone_config['name'], err))
        logs.loc[index,'Status'] = err


def remove_phone(service, phone_name, logs):
    try:
        service.removePhone(phone_name)
    except Fault as err:
        print("{} Phone deletion failed".format(phone_name))

def prod_process():
    try:
        if sys.argv[1]:
            input_data = read_input_data(sys.argv[1])
            phone_list = input_data['Device Name(Line)Sort Descending'].to_frame()
        else:
            print("Please specify the input csv file path.. e.g. python ucm-phone-migration.py test.csv")
            sys.exit(0)

        logs = phone_list.copy()
        logs['Status'] = np.nan
        for index, phone in phone_list:
            phone_config = get_phone(service, phone)
            new_phone = refine_phone_input(phone_config)
            #remove_phone(service, new_phone, index,logs)
            add_phone(service, new_phone,index,logs)
        logs.to_csv('logs.csv',index=False)
    except OSError as err:
        logs.loc[index, 'Status'] = err
        print("Error: {}".format(err))


In [25]:
input_data = read_input_data('inputs/HWLE 8945 Test.csv')
try:
    phone_list = input_data['Device Name(Line)Sort Descending'].to_frame()
    logs = phone_list.copy()
    logs['Status'] = np.nan
    for index, phone in phone_list.iterrows():
        phone_config = get_phone(service, phone['Device Name(Line)Sort Descending'])
        new_phone = refine_phone_input(phone_config)
        #remove_phone(service, new_phone, index,logs)
        add_phone(service, new_phone,index,logs)
    logs.to_csv('logs.csv',index=False)
except OSError as err:
        logs.loc[index, 'Status'] = err
        print("Error: {}".format(err))

/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '198.18.133.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '198.18.133.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


SEP123212321299 Add SIP Phone failed. Error: Could not insert new row - duplicate value in a UNIQUE INDEX column (Unique Index:).


/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '198.18.133.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


SEP123212321299 Add SIP Phone failed. Error: Could not insert new row - duplicate value in a UNIQUE INDEX column (Unique Index:).


/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '198.18.133.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [14]:
phone_list = input_data['Device Name(Line)Sort Descending'].to_frame()

In [15]:
phone_list

,Device Name(Line)Sort Descending
0,SEP1C36C79B25B9


In [16]:
service.getUser(userid='aperez')

/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '198.18.133.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{
    'return': {
        'user': {
            'firstName': 'Anita',
            'displayName': 'Anita Perez',
            'middleName': None,
            'lastName': 'Perez',
            'emMaxLoginTime': None,
            'userid': 'aperez',
            'password': None,
            'pin': None,
            'mailid': 'aperez@cb115.dc-02.com',
            'department': 'Sales',
            'manager': 'tbard',
            'userLocale': None,
            'associatedDevices': {
                'device': [
                    'CSFAPEREZ',
                    'SEPB1B1B1B1B1B1',
                    'SEPB2B2B2B2B2B2'
                ]
            },
            'primaryExtension': {
                'pattern': '6017',
                'routePartitionName': 'Base_PT'
            },
            'associatedPc': None,
            'associatedGroups': {
                'userGroup': [
                    {
                        'name': 'Standard CTI Enabled',
                        'userRoles': {

In [18]:
service.updateUser(userid='aperez', phoneProfiles='DP-aperez')

/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '198.18.133.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{
    'return': '{42B943AC-A016-EEC0-C887-528897BACB84}',
    'sequence': None
}

# Extension Mobility Test

In [1]:
import requests

In [49]:
url = "https://198.18.133.3:8443/emservice/EMServiceServlet"
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}

#payload='xml=%3Crequest%3E%0A%3CappInfo%3E%0A%3CappID%3Eadministrator%3C%2FappID%3E%0A%3CappCertificate%3EdCloud123!%3C%2FappCertificate%3E%0A%3C%2FappInfo%3E%0A%3Clogin%3E%0A%3CdeviceName%3ESEPF07816D1DAA2%3C%2FdeviceName%3E%0A%3CuserID%3Eaperez%3C%2FuserID%3E%0A%3CdeviceProfile%3EDP-Anita%3C%2FdeviceProfile%3E%0A%3CexclusiveDuration%3E%0A%3Ctime%3E60%3C%2Ftime%3E%0A%3C%2FexclusiveDuration%3E%0A%3C%2Flogin%3E%0A%3C%2Frequest%3E'
payload = '''xml=
<request>
<appInfo>
<appID>administrator</appID>
<appCertificate>dCloud123!</appCertificate>
</appInfo>
<login>
<deviceName>SEPB4AE2BC9C05A</deviceName>
<userID>aperez</userID>
<deviceProfile>DP-aperez</deviceProfile>
</login>
</request>
'''

# headers = {
#   'Content-Type': 'text/xml'
# }

response = requests.request("POST", url, headers=headers, data=payload,verify=False)

print(response.text)

/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '198.18.133.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<?xml version="1.0"?>
<response>
<success/>
</response>


In [47]:
import xmltodict, json
data = xmltodict.parse(response.text)
print(data)

{'response': {'failure': {'error': {'@code': '25', '#text': 'Policy Violation: com.cisco.emservice.PolicyValidatorException: Not allowed to log into multiple devices.'}}}}


In [40]:
payload = '''xml
<request>
<appInfo>
<appID>administrator</appID>
<appCertificate>dCloud123!</appCertificate>
</appInfo>
<login>
<deviceName>SEPB1B1B1B1B1B1</deviceName>
<userID>aperez</userID>
<deviceProfile>DP-Anita</deviceProfile>
<exclusiveDuration>
<time>60</time>
</exclusiveDuration>
</login>
</request>
'''


In [45]:
payload.encode()

b'xml\n<request>\n<appInfo>\n<appID>administrator</appID>\n<appCertificate>dCloud123!</appCertificate>\n</appInfo>\n<login>\n<deviceName>SEPF07816D1DAA2</deviceName>\n<userID>aperez</userID>\n<deviceProfile>DP-Anita</deviceProfile>\n<exclusiveDuration>\n<time>60</time>\n</exclusiveDuration>\n</login>\n</request>\n'

In [3]:
import pandas as pd
def read_input_data(file):
    data = pd.read_csv(file)
    return data

In [58]:
data = read_input_data('inputs/EM Logged in Detail test.csv').dropna()

In [56]:
logs = data.copy()
logs['Status'] = np.nan
data['response']['failure']['error']

nan


In [60]:
data

,First,Last,User ID,Device Name,Device Description,Remote Cluster ID,Time Stamp
0,sdf,dfd,aperez,SEPB4AE2BC9C05A,SEP64E950CA29CA,local,"Apr 19, 2021 6:20:45 AM"


In [6]:
def input_data_processing(data):
    return {
        'login_user':data['Device Name'],
        'login_dp':data['User ID']
    }



In [7]:
print(input_data_processing(data))

{'login_user': 0       SEP64E950CA29CA
1       SEP5CA48AFE7A69
2       SEP64E950CA0B58
3       SEPDCA5F43F29ED
4       SEPF02929584607
             ...       
1514    SEPC4143CB13FA7
1515    SEPC4143CB1400D
1516    SEP00192F7F28C4
1517    SEP6899CDA17839
1518    SEPE8EDF3A8CF5B
Name: Device Name, Length: 1519, dtype: object, 'login_dp': 0       2168
1       4601
2       6665
3       6508
4       5610
        ... 
1514    4989
1515    4606
1516    3682
1517    3866
1518    3475
Name: User ID, Length: 1519, dtype: int64}
